In [38]:
train_positive =  open("/data1/home/mukeshram/Natural_Language_Processing/Sentiment_analysis/dataset/Train.pos" , "r", encoding="latin-1").read()
train_negative =  open("/data1/home/mukeshram/Natural_Language_Processing/Sentiment_analysis/dataset/Train.neg" , "r", encoding="latin-1").read()
test_data =  open("/data1/home/mukeshram/Natural_Language_Processing/Sentiment_analysis/dataset/TestData" , "r", encoding="latin-1").read()

In [39]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [40]:
def clean(data):
    cleaned_data = []

    for i in data:
        i = i.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        i = pattern.sub('', i)
        i = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", i)
        tokens = word_tokenize(i)
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        words = [w for w in tokens if w.isalpha() and w not in stop_words]
        words = ' '.join(words)
        cleaned_data.append(words)
    return cleaned_data

In [41]:
train_pos = clean(train_positive.split("\n")[:-1])
train_neg = clean(train_negative.split("\n")[:-1])
test = clean(test_data.split("\n")[:-1])

In [42]:
train_neg[0]

'simplistic silly tedious'

In [43]:
x_train_p = [i +"1" for i in train_pos]
x_train_n = [i +"0" for i in train_neg]
x_data = x_train_p + x_train_n


import random
random.seed(42)
random.shuffle(x_data)


In [44]:
x_data[0:5]

['questioning election process payami graphically illustrates problems fledgling democracies also strength sense freedom iranian people already possess without access ballot box1',
 'bright side contains jesse ventura best work since xfl0',
 'acting particularly tambor almost makes never worthwhile writerdirector schaeffer follow titular advice0',
 'andersson creates world surreal disturbingly familiar absurd yet tremendously sad1',
 'always appreciated smartly written motion picture whatever flaws igby goes may possess undeniably1']

In [45]:
x_train_data = [i[:-1] for i in x_data]
y_train = [int(i[-1]) for i in x_data]

In [46]:
from gensim.models import KeyedVectors
word2vec_model = KeyedVectors.load("/data1/home/mukeshram/NLP/word2vec_google_news_model")

In [47]:
import numpy as np

def sentence_embedding(sentence, word2vec_model):
    words = sentence.split()
    sentence_vector_w2v = np.zeros(word2vec_model.vector_size)
    
    for word in words:
        if word in word2vec_model:
            sentence_vector_w2v += word2vec_model[word]

    num_words = len(words)
    if num_words > 0:
        sentence_vector_w2v /= num_words
    
    return sentence_vector_w2v


In [48]:
len(sentence_embedding("king is ok", word2vec_model))

300

In [49]:
x_train=[]
for i in x_train_data:
    x_train.append(sentence_embedding(i, word2vec_model))

In [50]:
x_test=[]
for i in test:
    x_test.append(sentence_embedding(i, word2vec_model))


In [51]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)

In [52]:
x_train.shape, y_train.shape

((10000, 300), (10000,))

In [53]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")

assert x_train.shape[0] == y_train.shape[0], "Number of samples in x_train and y_train must match."

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


x_train.shape: (10000, 300)
y_train.shape: (10000,)


Accuracy: 76.60%


In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [32]:
from sklearn.model_selection import cross_validate, train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.20, shuffle=False)


In [ ]:
from gensim.models import KeyedVectors
glove_model = KeyedVectors.load("/data1/home/mukeshram/NLP/glove-wiki-gigaword-300")

In [ ]:
len(word2vec_model["king"]),len(glove_model["king"])



In [ ]:
import numpy as np

def sentence_embedding(sentence, word2vec_model, glove_model):
    words = sentence.split()
    sentence_vector_w2v = np.zeros(word2vec_model.vector_size)
    sentence_vector_glove = np.zeros(glove_model.vector_size)
    
    for word in words:
        if word in word2vec_model:
            sentence_vector_w2v += word2vec_model[word]
        
        if word in glove_model:
            sentence_vector_glove += glove_model[word]
    
    num_words = len(words)
    if num_words > 0:
        sentence_vector_w2v /= num_words
        sentence_vector_glove /= num_words
    
    sentence_vector = np.concatenate((sentence_vector_w2v, sentence_vector_glove))
    return sentence_vector



In [ ]:
len(sentence_embedding("king", word2vec_model, glove_model))

In [ ]:
x_train=[]
for i in x_train_data:
    x_train.append(sentence_embedding(i, word2vec_model, glove_model))
    
    

In [ ]:
len(x_train)

In [ ]:
y_train[0]


In [ ]:
x_test=[]
for i in test:
    x_test.append(sentence_embedding(i, word2vec_model, glove_model))


In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.model_selection import cross_validate, train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.20, shuffle=False)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
model1 = keras.Sequential([
    layers.Input(shape=(600,)),  # Input layer
    layers.Dense(1200, activation='relu'),  # Hidden layer(s)
    layers.Dense(600, activation='relu'),  # Hidden layer(s)
    layers.Dense(300, activation='relu'),  # Hidden layer(s)

    layers.Dense(100, activation='relu'),  # Hidden layer(s)

    layers.Dense(1, activation='sigmoid')  # Output layer
])


In [ ]:
learning_rate = 0.001  # You can adjust this value
optimizer = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
model1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model1.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
history = model1.fit(X_train, y_train,
                    epochs=100,
                    # batch_size=16,
                    validation_split=0.2,
                    callbacks=[early_stopping],  # Use the early stopping callback
                    verbose=1)


In [ ]:
import matplotlib.pyplot as plt

test_loss, test_accuracy = model1.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'],marker = 'o')
plt.plot(history.history['val_accuracy'],marker = 'o')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')


# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'],marker = 'o')
plt.plot(history.history['val_loss'],marker = 'o')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
test_loss, test_accuracy = model1.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Indicate the point where early stopping occurred
best_val_loss_index = np.argmin(history.history['val_loss'])
plt.scatter(best_val_loss_index, history.history['val_accuracy'][best_val_loss_index], color='red', marker='o', label='Early Stopping')
plt.legend()

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

# Indicate the point where early stopping occurred
plt.scatter(best_val_loss_index, history.history['val_loss'][best_val_loss_index], color='red', marker='o', label='Early Stopping')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


model_lr = LogisticRegression()

model_lr.fit(X_train, y_train)

predictions = model_lr.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score



# Create SVM models with different kernels
linear_model = SVC(kernel='linear')
rbf_model = SVC(kernel='rbf')
poly_model = SVC(kernel='poly', degree=3)  # You can adjust the degree parameter

# Train the models on the training data
linear_model.fit(X_train, y_train)
rbf_model.fit(X_train, y_train)
poly_model.fit(X_train, y_train)

# Make predictions on the test data
linear_predictions = linear_model.predict(X_test)
rbf_predictions = rbf_model.predict(X_test)
poly_predictions = poly_model.predict(X_test)

# Calculate accuracies
linear_accuracy = accuracy_score(y_test, linear_predictions)
rbf_accuracy = accuracy_score(y_test, rbf_predictions)
poly_accuracy = accuracy_score(y_test, poly_predictions)

print(f"Linear Kernel Accuracy: {linear_accuracy:.2f}")
print(f"RBF Kernel Accuracy: {rbf_accuracy:.2f}")
print(f"Poly Kernel Accuracy: {poly_accuracy:.2f}")


In [ ]:
import tensorflow as tf

embedding_dim = 600
hidden_units = 128
num_classes = 2  # Replace with your actual number of classes

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(embedding_dim,)),
    tf.keras.layers.Reshape(target_shape=(1, embedding_dim)),
    tf.keras.layers.SimpleRNN(hidden_units),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='Adagrad', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=500,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping],  # Use the early stopping callback
                    verbose=1)

In [ ]:
import matplotlib.pyplot as plt

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'],marker = 'o')
plt.plot(history.history['val_accuracy'],marker = 'o')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')


# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'],marker = 'o')
plt.plot(history.history['val_loss'],marker = 'o')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf

embedding_dim = 600
hidden_units = 128
num_classes = 10  # Replace with your actual number of classes

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(embedding_dim,)),
    tf.keras.layers.Reshape(target_shape=(1, embedding_dim)),
    tf.keras.layers.LSTM(hidden_units),  # Change SimpleRNN to LSTM
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='Adagrad', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=500,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping],  # Use the early stopping callback
                    verbose=1)

In [ ]:
import matplotlib.pyplot as plt

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'],marker = 'o')
plt.plot(history.history['val_accuracy'],marker = 'o')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')


# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'],marker = 'o')
plt.plot(history.history['val_loss'],marker = 'o')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf

embedding_dim = 600
hidden_units = 128
num_classes = 10  # Replace with your actual number of classes

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(embedding_dim,)),
    tf.keras.layers.Reshape(target_shape=(1, embedding_dim)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1200, return_sequences=True)),  # Bidirectional LSTM
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(300, return_sequences=True)),   # Bidirectional LSTM
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=False)),  # Bidirectional LSTM
    tf.keras.layers.Dense(1200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='Adagrad', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=500,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping],  # Use the early stopping callback
                    verbose=1)

In [ ]:
import matplotlib.pyplot as plt

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'],marker = 'o')
plt.plot(history.history['val_accuracy'],marker = 'o')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')


# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'],marker = 'o')
plt.plot(history.history['val_loss'],marker = 'o')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()